# Roman WFI Synthetic Photometry with synphot

***

## Kernel Information

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

## Imports
- *os* for access to environment variables and file paths
- *astropy.units* for unit specification and conversion
- *matplotlib.pyplot* for plotting
- *numpy* for data array creation and manipulation
- *synphot* for synthetic photometry
- *stsynphot* for access to HST, Roman, and other transmission curves
- *webbpsf* for access to Roman and JWST transmission curves (optional)

In [ ]:
%matplotlib inline
import os

from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np

import synphot as syn
import stsynphot as stsyn
#import webbpsf

## Introduction
In this tutorial, we will demonstrate how to use synthetic photometry software to estimate the brightness of both empirical and theoretical sources as they would be measured by the Roman Wide Field Instrument (WFI). Note that we will use terms such as transmission and throughput interchangeably throughout the tutorial, but what we mean is the unitless measurement efficiency as a function of wavelength (or frequency) for the detection of photons at the end of the optics chain between the Roman primary mirror and the WFI detectors including all components in between (e.g., secondary mirror, optical elements, etc.).

***

## Loading data
Data are loaded from Flexible Image Transport System (FITS) binary tables using functions within `stsynphot`. Data may also be loaded as NumPy NDArray objects, or for the Roman WFI and James Webb Space Telescope (JWST) instruments may be retrieved from `webbpsf`. Note that Roman WFI throughput information in `stsynphot` and `webbpsf` produce identical results. JWST instrument throughputs are currently only available via `webbpsf`. 

**NOTE:** Roman WFI throughputs are currently for a single detector and combine the entire optical chain. Updates to the transmission information are expected as part of the ground testing campaign of both the WFI and the integrated observatory.

## Use Cases and Examples

### Plot the Roman WFI Filter Throughputs and Information

There are two approaches to retrieving WFI throughput information. Here we will show both methods of loading the throughput information. All the remaining steps in the tutorial are independent from the specific loading method.

#### Retrieving WFI Throughputs from WebbPSF

To retrieve the optical element throughput information from `webbpsf`, we set up the WFI object, and use the method _get_synphot_bandpass, which takes the optical element name as the only argument. In the example below, we load the throughput information from the F129 imaging filter. This new Python object behaves as a callable function that takes as input the wavelength or frequency of interest, and will return the throughput at that wavelength or frequency.

Note that we have commented out the lines in the following cell as this is optional for Roman, but is required for JWST.

In [ ]:
# Set up the Roman WFI object and retrieve the throughput of a filter
#roman = webbpsf.WFI()
#wfi_f129 = roman._get_synphot_bandpass('F129')
#print(wfi_f129(1.29 * u.micron))

#### Retrieving WFI Throughputs from stsynphot

If we want to retrieve the same information using stsynphot, which contains tools to load STScI-specific bandpass information from the Spectral Atlas Files for Synphot Software, we use the synphot.band class and give as input a string that contains the mission, instrument, and element name, e.g., "roman, wfi, f129":

In [ ]:
# Set up the Roman WFI object and retrieve the throughput of a filter
wfi_f129 = stsyn.band('roman, wfi, f129')
print(wfi_f129(1.29 * u.micron))

#### Plotting and Computing Bandpass Information

Regardless of the method we chose to load the throughput data, we can then plot the throughput curves and compute some information about our bandpasses. The following code block will generate a plot of the WFI imaging filter throughputs (with the exception of the wide F146 filter, which has been omitted for clarity):

In [ ]:
# Set up the Roman WFI object and make a list of the optical element names
roman_filter = 'F062 F087 F106 F129 F158 F184 F213'.lower().split()
 
# Set up wavelengths from 0.4 to 2.5 microns in increments of 0.01 microns.
waves = np.arange(0.4, 2.5, 0.01) * u.micron
 
# Set up figure
fig, ax = plt.subplots()
 
# For each optical element, plot the throughput in a different color
# and shade the area below the curve.
colors = plt.cm.rainbow(np.linspace(0, 1, len(roman_filter)))
for i, f in enumerate(roman_filter):
    band = stsyn.band(f'roman,wfi,{f}')
    clean = np.where(band(waves) > 0)
    ax.plot(waves[clean], band(waves[clean]), color=colors[i])
    ax.fill_between(waves[clean].value, band(waves[clean]).value, alpha=0.5, color=colors[i])
 
# Set plot axis labels, ranges, and add grid lines
ax.set_xlabel('Wavelength ($\mu$m)')
ax.set_ylabel('Throughput')
ax.set_ylim(0, 1.05)
ax.set_xlim(0.4, 2.5)
ax.grid(':', alpha=0.3)

We can also get information about the bandpasses using the following methods on the bandpass objects in synphot:

In [ ]:
band = stsyn.band('roman, wfi, f129')
print('WFI F129:')
print(f'\tBandwidth: {band.photbw():.5f}')
print(f'\tPivot wavelength: {band.pivot():.5f}')
print(f'\tFWHM: {band.fwhm():.5f}')

### Synthetic Photometry of a Simulated Source

Here we show how to perform synthetic photometry of a simulated source. For this example, we will use a star of spectral type M5III, and we will add additional extinction to the source spectrum using the Milky Way extinction law. We also normalize the spectrum to a particular Johnson V-band AB magnitude, but this could be normalized to any flux density or equivalent in any band (e.g., Hubble and JWST instruments, 2MASS, etc.).

In [ ]:
# Load the WFI F158 element
wfi_f158 = stsyn.band('roman, wfi, f158')
 
# Retrieve the spectrum of the star
star_spec = syn.SourceSpectrum.from_file(os.path.join(os.environ['PYSYN_CDBS'], 'grid', 'pickles', 'dat_uvk', 'pickles_uk_100.fits'))
 
# Renormalize the spectrum to have a Johnson V-band magnitude of 22.5 ABmag
norm_spec = star_spec.normalize(22.5 * u.ABmag, band=stsyn.band('johnson, v'))
 
# Add extinction corresponding to E(B-V) = 2.3 using the Milky Way extinction law (Rv = 3.1)
extinction = syn.ReddeningLaw.from_extinction_model('mwavg').extinction_curve(2.3)
final_spec = norm_spec * extinction
 
# Compute the flux density arrays
wavelengths = np.arange(0.5, 2.5, 0.001) * u.micron
norm_fluxes = norm_spec(wavelengths, flux_unit=u.Jy)
final_fluxes = final_spec(wavelengths, flux_unit=u.Jy)
 
# Plot the spectra
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(wavelengths, norm_fluxes, color='blue', lw=2, label='Normalized Spectrum, V = 22.5 ABmag')
ax.plot(wavelengths, final_fluxes, color='red', lw=2, label=r'Reddened Spectrum, E(B $-$ V) = 2.3')
ax.legend(loc=2)
ax.set_ylabel('Flux Density (Jy)')
ax.set_xlabel(r'Wavelength ($\mu$m)');

Now that we have our spectrum, we can "observe" it using the WFI:

In [ ]:
# Observe the spectrum with the F158 bandpass
obs_spec = syn.Observation(final_spec, wfi_f158)
 
# Print out the magnitudes, flux density, and count rate (electrons per second) of the observed source
print('M5III star, WFI F158:')
print(f'\t{obs_spec.effstim(syn.units.VEGAMAG, vegaspec=syn.SourceSpectrum.from_file(syn.conf.vega_file))}')
print(f'\t{obs_spec.effstim(u.ABmag)}')
print(f'\t{obs_spec.effstim(u.Jy)}')
print(f'\t{obs_spec.countrate(area=np.pi * (2.4 * u.m)**2)}')

Now, let's suppose that we wish to estimate the brightness of a source with a spectral energy distribution that can be described with a power law with spectral index of $\alpha=-1$. We have access to a library of Astropy models that can be used to approximate the input spectrum. We will normalize our example to a flux density of 5 mJy at 1.5 $\mu$m.

In [ ]:
# Create spectrum model
source = syn.SourceSpectrum(syn.models.PowerLawFlux1D(5 * u.mJy, 1.5 * u.micron, -1))

Now, let's plot the spectrum and make sure that it looks correct:

In [ ]:
# Create a wavelength array from 0.6 to 2.5 microns in bins of 0.001 micron.
waves = np.arange(0.6, 2.5, 0.001) * u.micron

# Plot the model
fig, ax = plt.subplots()
ax.plot(waves, source(waves, flux_unit=u.Jy))
ax.set_ylabel('Flux Density (Jy)')
ax.set_xlabel(r'Wavelength ($\mu$m)')
ax.set_yscale('log');

Let's add another component to our spectrum to see how models can be combined. In this case, let's add a Gaussian absorption line at 1.2 $\mu$m with an amplitude of 2 mJy and a full-width half-maximum (FWHM) of 0.1 $\mu$m, and a Gaussian emission line at 1.5 $\mu$m with an amplitude of 1 mJy and a FWHM of 0.01 $\mu$m.

In [ ]:
# Create Gaussian emission lines
abs_line = syn.SourceSpectrum(syn.models.GaussianFlux1D, amplitude = 2 * u.mJy, mean = 1.2 * u.micron, fwhm = 0.1 * u.micron)
em_line = syn.SourceSpectrum(syn.models.GaussianFlux1D, amplitude = 1 * u.mJy, mean = 1.5 * u.micron, fwhm = 0.01 * u.micron)

# Add components together and plot the total model
total = source - abs_line + em_line
fig, ax = plt.subplots()
ax.plot(waves, total(waves, flux_unit=u.Jy))
ax.set_xlabel(r'Wavelength ($\mu$m)')
ax.set_ylabel('Flux Density (Jy)')
ax.set_yscale('log');

Let's observe this model spectrum and take a look at what it looks like after it passes through the optical chain by convolving the model with one of our transmission curves:

In [ ]:
f129 = stsyn.band('roman,wfi,f129')
obs_model = syn.Observation(total, f129)

# Add components together and plot the total model
fig, ax = plt.subplots()
ax.plot(waves, obs_model(waves, flux_unit=u.Jy))
ax.set_xlabel(r'Wavelength ($\mu$m)')
ax.set_ylabel('Observed Flux Density (Jy)');

With our observation object, we can get the integrated flux just as we did before:

In [ ]:
print(f'{obs_model.effstim(u.Jy)}')

## Aditional Resources

- [synphot API documentation](https://synphot.readthedocs.io/en/latest/)
- [stsynphot API documentation](https://stsynphot.readthedocs.io/en/latest/)
- [Reference Atlases](https://archive.stsci.edu/hlsp/reference-atlases)
- [RDox Synphot for Roman Article](https://roman-docs.stsci.edu/simulation-tools-handbook-home/simulation-development-utilities/synphot-for-roman)

## About this notebook

**Author:** Tyler Desjardins  
**Updated On:** 2024-05-07

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 